In [2]:
from attrs import asdict, define, make_class, Factory

@define
class SomeClass:
    a_number: int = 42
    list_of_numbers: list[int] = Factory(list)

    def hard_math(self, another_number):
        return self.a_number + sum(self.list_of_numbers) * another_number


In [3]:

sc = SomeClass(1, [1, 2, 3])


In [4]:
sc

SomeClass(a_number=1, list_of_numbers=[1, 2, 3])

In [5]:
sc.hard_math(5)

31

In [6]:
d = asdict(sc)

In [7]:
d

{'a_number': 1, 'list_of_numbers': [1, 2, 3]}

So, attrs does:
- Takes the class definition, 
- writes the dunder methods based on that information and attaches them to the classes. 
  

### How much boilerplate does this save?

#### `attrs` approach

In [9]:
from attrs import define, field

@define
class SmartClass:
   a = field()
   b = field()
SmartClass(1, 2)

SmartClass(a=1, b=2)

#### class with boilerplate

In [10]:
class ArtisanalClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __repr__(self):
        return f"ArtisanalClass(a={self.a}, b={self.b})"

    def __eq__(self, other):
        if other.__class__ is self.__class__:
            return (self.a, self.b) == (other.a, other.b)
        else:
            return NotImplemented

    def __ne__(self, other):
        result = self.__eq__(other)
        if result is NotImplemented:
            return NotImplemented
        else:
            return not result

    def __lt__(self, other):
        if other.__class__ is self.__class__:
            return (self.a, self.b) < (other.a, other.b)
        else:
            return NotImplemented

    def __le__(self, other):
        if other.__class__ is self.__class__:
            return (self.a, self.b) <= (other.a, other.b)
        else:
            return NotImplemented

    def __gt__(self, other):
        if other.__class__ is self.__class__:
            return (self.a, self.b) > (other.a, other.b)
        else:
            return NotImplemented

    def __ge__(self, other):
        if other.__class__ is self.__class__:
            return (self.a, self.b) >= (other.a, other.b)
        else:
            return NotImplemented

    def __hash__(self):
        return hash((self.__class__, self.a, self.b))
    
    
    
ArtisanalClass(a=1, b=2)

ArtisanalClass(a=1, b=2)

That’s quite a mouthful and it doesn’t even use any of attrs’s more advanced features like validators or default values. 

Also: no tests whatsoever. And who will guarantee you, that you don’t accidentally flip the < in your tenth implementation of __gt__?

> It also should be noted that attrs is not an all-or-nothing solution. You can freely choose which features you want and disable those that you want more control over:

In [11]:
@define
class SmartClass:
   a: int
   b: int

   def __repr__(self):
       return f"<SmartClass(a={self.a})>"

In [13]:
SmartClass(1,2)

<SmartClass(a=1)>

In [14]:
sc_1 = SmartClass(1,2)
sc_2 = SmartClass(2,3)

In [15]:
sc_1 == sc_2

False

In [17]:
sc_1 > sc_2

TypeError: '>' not supported between instances of 'SmartClass' and 'SmartClass'

It seems by default attrs will not implement the __lt__, __gt__, __le__, __ge__ methods.

- Adding the `order` parameter to the `define` decorator will make the lt and other methods work as expected.
- so we will need to enable the ordering.

In [18]:

@define(order=True)
class SomeClass:
    x = field()
    y = field()

sc_1 = SomeClass(1, 2)
sc_2 = SomeClass(3, 4)

In [19]:

sc_1 < sc_2

True